In [1]:
!pip install neuralforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.8/722.8 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.7 MB/s eta 0:00:00


In [2]:
!pip install ripser

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ripser: filename=ripser-0.6.4-cp310-cp310-linux_x86_64.whl size=752978 sha256=20a4db96c418480fb0cb4911e1e3f986e2b66059b830d91465297f36dd4baffb
  Stored in directory: /root/.cache/pip/wheels/c5/f5/66/f41f708b049057431155934f74e20ca6001a085fcd2e615150
  Created wheel for hopcroftkarp: filename=hopcroftkarp-1.2.5-py2.py3-none-any.whl size=18101 sha256=783bf0d332f99dd6ff0daa76954d209470f9855b173ebbe079c714ce3c64032d
  Stored in directory: /root/.cache/pip/wheels/ef/0f/3b/0f931844eecc34addd90e72d54cd39c08b7066c5f25c00b9a4
Successfully built ripser hopcroftkarp


In [1]:
from ripser import Rips
import persim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import torch
from typing import Optional, Union, Tuple


import torch.nn.functional as F
from torch.distributions import Distribution
from torch.distributions import Bernoulli, Normal, StudentT, Poisson, NegativeBinomial

from torch.distributions import constraints


from neuralforecast import NeuralForecast
from neuralforecast.models import TFT
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic
from neuralforecast.losses.numpy import rmse, mape
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

In [3]:
from numpy.random import seed
from random import randrange

In [4]:
def _divide_no_nan(a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
    """
    Auxiliary funtion to handle divide by 0
    """
    div = a / b
    div[div != div] = 0.0
    div[div == float("inf")] = 0.0
    return div

def _weighted_mean(losses, weights):
    """
    Compute weighted mean of losses per datapoint.
    """
    return _divide_no_nan(torch.sum(losses * weights), torch.sum(weights))

# %% ../../nbs/losses.pytorch.ipynb 7
class BasePointLoss(torch.nn.Module):
    """
    Base class for point loss functions.

    **Parameters:**<br>
    `horizon_weight`: Tensor of size h, weight for each timestamp of the forecasting window. <br>
    `outputsize_multiplier`: Multiplier for the output size. <br>
    `output_names`: Names of the outputs. <br>
    """

    def __init__(self, horizon_weight, outputsize_multiplier, output_names):
        super(BasePointLoss, self).__init__()
        if horizon_weight is not None:
            horizon_weight = torch.Tensor(horizon_weight.flatten())
        self.horizon_weight = horizon_weight
        self.outputsize_multiplier = outputsize_multiplier
        self.output_names = output_names
        self.is_distribution_output = False

    def domain_map(self, y_hat: torch.Tensor):
        """
        Univariate loss operates in dimension [B,T,H]/[B,H]
        This changes the network's output from [B,H,1]->[B,H]
        """
        return y_hat.squeeze(-1)

    def _compute_weights(self, y, mask):
        """
        Compute final weights for each datapoint (based on all weights and all masks)
        Set horizon_weight to a ones[H] tensor if not set.
        If set, check that it has the same length as the horizon in x.
        """
        if mask is None:
            mask = torch.ones_like(y).to(y.device)

        if self.horizon_weight is None:
            self.horizon_weight = torch.ones(mask.shape[-1])
        else:
            assert mask.shape[-1] == len(
                self.horizon_weight
            ), "horizon_weight must have same length as Y"

        weights = self.horizon_weight.clone()
        weights = torch.ones_like(mask, device=mask.device) * weights.to(mask.device)
        return weights * mask


class MSE_2DWD(BasePointLoss):
    from ripser import Rips
    import persim

    def __init__(self, horizon_weight=None):
        super(MSE_2DWD, self).__init__(
            horizon_weight=horizon_weight, outputsize_multiplier=1, output_names=[""]
        )

    def __call__(
        self,
        y: torch.Tensor,
        y_hat: torch.Tensor,
        mask: Union[torch.Tensor, None] = None,
    ):
        """
        **Parameters:**<br>
        `y`: tensor, Actual values.<br>
        `y_hat`: tensor, Predicted values.<br>
        `mask`: tensor, Specifies datapoints to consider in loss.<br>

        **Returns:**<br>
        `mse`: tensor (single value).
        """
        rips = Rips(maxdim = 2, verbose=False)
        n=2
        wasserstein_dists = np.zeros((n))

        for i in range(2):
          dgm1 = rips.fit_transform(y_hat[:,int(y_hat.shape[1]/2*i):int(y_hat.shape[1]/2*(i+1))].to('cpu').detach().numpy())
          dgm2 = rips.fit_transform(y[:,int(y_hat.shape[1]/2*i):int(y_hat.shape[1]/2*(i+1))].to('cpu').detach().numpy())
          wasserstein_dists[i] = persim.wasserstein(dgm1[0], dgm2[0], matching=False)

        losses = torch.sum((y - y_hat) ** 2)/(y_hat.shape[0]*y_hat.shape[1]) + torch.tensor((np.sum(wasserstein_dists))/n)
        return losses

In [5]:
df = pd.read_excel("SP100_vol.xlsx", index_col="Date",parse_dates=True)
columns=df.columns
df_static=pd.read_excel("Static_df.xlsx")
n_series=len(df.columns)
train = df.iloc[:int(len(df['AMT'])*0.7)]
validation_length=int(len(df['AMT'])*0.7)-int(len(df['AMT'])*0.5)
train=train.unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

In [6]:
df.columns

Index(['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
      dtype='object')

In [7]:
n_inputs = [3,5,10,15,21,42,84]
hidden_size = [50,100,150,200,300,500]
epochs=[50,100,150,250,350,450,550,650,750]
dropout=[0,0.2,0.3,0.4]
n_head=[2,4,5,8,10,20]
attn_dropout=[0,0.2,0.3,0.4]
learning_rate=[0.0005,0.0001,0.00005,0.00001]
num_lr_decays=[5,3,2,1]
scaler_type=["robust","standard",'minmax']

# Hyperparameter Search

In [ ]:
for m in range (5):
  i=randrange(7)
  h=randrange(6)
  a=randrange(4)
  a1=randrange(4)
  a2=randrange(4)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)

  model=TFT(h=10, input_size=n_inputs[i], stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=hidden_size[h],
       n_head=n_head[h], attn_dropout=attn_dropout[a1], dropout=dropout[a2], loss=MSE_2DWD(),
       max_steps=epochs[k], learning_rate=learning_rate[n],val_check_steps=700,
       num_lr_decays=num_lr_decays[n],
       step_size=10,
       scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=10,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%10,step_size=10)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | Hidden Size: {hidden_size[h]} | Scaler Type: {scaler_type[o]}")
  print(f"N head: {n_head[h]} | Learning Rate: {n} | Attn Droupout: {attn_dropout[a1]} | Droupout: {dropout[a2]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE}")

INFO:lightning_fabric.utilities.seed:Global seed set to 54387889


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

...

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Epoch: 250 | Inputs: 15 | Hidden Size: 500 | Scaler Type: robust
N head: 20 | Learning Rate: 0 | Attn Droupout: 0 | Droupout: 0.3
RMSE: 0.0036163746912893 | QLIKE: 0.04671734856630836 | MAE: 0.0023817746964641976


In [ ]:
for m in range (5):
  i=randrange(7)
  h=randrange(6)
  a=randrange(4)
  a1=randrange(4)
  a2=randrange(4)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)

  model=TFT(h=10, input_size=n_inputs[i], stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=hidden_size[h],
       n_head=n_head[h], attn_dropout=attn_dropout[a1], dropout=dropout[a2], loss=MSE_2DWD(),
       max_steps=epochs[k], learning_rate=learning_rate[n],val_check_steps=700,
       num_lr_decays=num_lr_decays[n],
       step_size=10,
       scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=10,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%10,step_size=10)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | Hidden Size: {hidden_size[h]} | Scaler Type: {scaler_type[o]}")
  print(f"N head: {n_head[h]} | Learning Rate: {n} | Attn Droupout: {attn_dropout[a1]} | Droupout: {dropout[a2]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE}")

INFO:lightning_fabric.utilities.seed:Global seed set to 6054677


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Epoch: 650 | Inputs: 3 | Hidden Size: 200 | Scaler Type: standard
N head: 8 | Learning Rate: 3 | Attn Droupout: 0.2 | Droupout: 0.3
RMSE: 0.004413218319188793 | QLIKE: 0.05862614928580061 | MAE: 0.002942944166251971


In [ ]:
for m in range (5):
  i=randrange(7)
  h=randrange(6)
  a=randrange(4)
  a1=randrange(4)
  a2=randrange(4)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)

  model=TFT(h=10, input_size=n_inputs[i], stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=hidden_size[h],
       n_head=n_head[h], attn_dropout=attn_dropout[a1], dropout=dropout[a2], loss=MSE_2DWD(),
       max_steps=epochs[k], learning_rate=learning_rate[n],
       num_lr_decays=num_lr_decays[n],val_check_steps=700,
       step_size=10,
       scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=10,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%10,step_size=10)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | Hidden Size: {hidden_size[h]} | Scaler Type: {scaler_type[o]}")
  print(f"N head: {n_head[h]} | Learning Rate: {n} | Attn Droupout: {attn_dropout[a1]} | Droupout: {dropout[a2]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE}")

INFO:lightning_fabric.utilities.seed:Global seed set to 2097356


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

...

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Epoch: 150 | Inputs: 5 | Hidden Size: 300 | Scaler Type: standard
N head: 10 | Learning Rate: 1 | Attn Droupout: 0.4 | Droupout: 0.4
RMSE: 0.004266291381517094 | QLIKE: 0.05589390044845217 | MAE: 0.0028234902398685258


In [ ]:
for m in range (5):
  i=randrange(7)
  h=randrange(6)
  a=randrange(4)
  a1=randrange(4)
  a2=randrange(4)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)

  model=TFT(h=10, input_size=n_inputs[i], stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=hidden_size[h],
       n_head=n_head[h], attn_dropout=attn_dropout[a1], dropout=dropout[a2], loss=MSE_2DWD(),
       max_steps=epochs[k], learning_rate=learning_rate[n],val_check_steps=700,
       num_lr_decays=num_lr_decays[n],
       step_size=10,
       scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=10,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%10,step_size=10)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | Hidden Size: {hidden_size[h]} | Scaler Type: {scaler_type[o]}")
  print(f"N head: {n_head[h]} | Learning Rate: {n} | Attn Droupout: {attn_dropout[a1]} | Droupout: {dropout[a2]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE}")

INFO:lightning_fabric.utilities.seed:Global seed set to 72076149


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

...

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Epoch: 550 | Inputs: 42 | Hidden Size: 200 | Scaler Type: minmax
N head: 8 | Learning Rate: 0 | Attn Droupout: 0.2 | Droupout: 0
RMSE: 0.003709247627146767 | QLIKE: 0.04884355361710147 | MAE: 0.0024762624505948556


In [ ]:
for m in range (5):
  i=randrange(7)
  h=randrange(6)
  a=randrange(4)
  a1=randrange(4)
  a2=randrange(4)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)

  model=TFT(h=10, input_size=n_inputs[i], stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=hidden_size[h],
       n_head=n_head[h], attn_dropout=attn_dropout[a1], dropout=dropout[a2], loss=MSE_2DWD(),
       max_steps=epochs[k], learning_rate=learning_rate[n],val_check_steps=700,
       num_lr_decays=num_lr_decays[n],
       step_size=10,
       scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=10,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%10,step_size=10)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | Hidden Size: {hidden_size[h]} | Scaler Type: {scaler_type[o]}")
  print(f"N head: {n_head[h]} | Learning Rate: {n} | Attn Droupout: {attn_dropout[a1]} | Droupout: {dropout[a2]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE}")

INFO:lightning_fabric.utilities.seed:Global seed set to 113650933


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

...

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Epoch: 450 | Inputs: 84 | Hidden Size: 300 | Scaler Type: robust
N head: 10 | Learning Rate: 0 | Attn Droupout: 0.3 | Droupout: 0
RMSE: 0.0037641093623733005 | QLIKE: 0.04928473162088479 | MAE: 0.0024053244274671527


In [ ]:
for m in range (5):
  i=randrange(7)
  h=randrange(6)
  a=randrange(4)
  a1=randrange(4)
  a2=randrange(4)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)

  model=TFT(h=10, input_size=n_inputs[i], stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=hidden_size[h],
       n_head=n_head[h], attn_dropout=attn_dropout[a1], dropout=dropout[a2], loss=MSE_2DWD(),
       max_steps=epochs[k], learning_rate=learning_rate[n],val_check_steps=700,
       num_lr_decays=num_lr_decays[n],
       step_size=10,
       scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=10,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%10,step_size=10)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | Hidden Size: {hidden_size[h]} | Scaler Type: {scaler_type[o]}")
  print(f"N head: {n_head[h]} | Learning Rate: {n} | Attn Droupout: {attn_dropout[a1]} | Droupout: {dropout[a2]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE}")

INFO:lightning_fabric.utilities.seed:Global seed set to 55186135


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

...

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Epoch: 150 | Inputs: 21 | Hidden Size: 300 | Scaler Type: minmax
N head: 10 | Learning Rate: 2 | Attn Droupout: 0.4 | Droupout: 0.3
RMSE: 0.003818746786516982 | QLIKE: 0.052141689342098695 | MAE: 0.0026045957807328403


In [ ]:
for m in range (5):
  i=randrange(7)
  h=randrange(6)
  a=randrange(4)
  a1=randrange(4)
  a2=randrange(4)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)

  model=TFT(h=10, input_size=n_inputs[i], stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=hidden_size[h],
       n_head=n_head[h], attn_dropout=attn_dropout[a1], dropout=dropout[a2], loss=MSE_2DWD(),
       max_steps=epochs[k], learning_rate=learning_rate[n],val_check_steps=700,
       num_lr_decays=num_lr_decays[n],
       step_size=10,
       scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=10,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%10,step_size=10)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | Hidden Size: {hidden_size[h]} | Scaler Type: {scaler_type[o]}")
  print(f"N head: {n_head[h]} | Learning Rate: {n} | Attn Droupout: {attn_dropout[a1]} | Droupout: {dropout[a2]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE}")

INFO:lightning_fabric.utilities.seed:Global seed set to 79285569


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

...

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Epoch: 550 | Inputs: 3 | Hidden Size: 200 | Scaler Type: robust
N head: 8 | Learning Rate: 2 | Attn Droupout: 0 | Droupout: 0
RMSE: 0.0053273853201128915 | QLIKE: 0.09470903198318824 | MAE: 0.003376858773688992


INFO:lightning_fabric.utilities.seed:Global seed set to 85595024

Epoch: 250 | Inputs: 84 | Hidden Size: 300 | Scaler Type: standard
N head: 10 | Learning Rate: 3 | Attn Droupout: 0.3 | Droupout: 0.4
RMSE: 0.003562362181179557 | QLIKE: 0.046427722110966935 | MAE: 0.002332665936958569

In [ ]:
for m in range (5):
  randomseed=randrange(129228148)
  model=TFT(h=10, input_size=84, stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=300,
       n_head=10, attn_dropout=0.3, dropout=0.4, loss=MSE_2DWD(),
       max_steps=250, learning_rate=learning_rate[3],val_check_steps=700,
       num_lr_decays=num_lr_decays[3],
       step_size=10,
       scaler_type='standard', random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=10,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%10,step_size=10)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE}")

INFO:lightning_fabric.utilities.seed:Global seed set to 29759059


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

...

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Random Seed: 52330090
RMSE: 0.0035334524242688387 | QLIKE: 0.04587371219157712 | MAE: 0.002318323533179664


In [ ]:
for m in range (5):
  randomseed=randrange(129228148)
  model=TFT(h=10, input_size=84, stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=300,
       n_head=10, attn_dropout=0.3, dropout=0.4, loss=MSE_2DWD(),
       max_steps=250, learning_rate=learning_rate[3],val_check_steps=700,
       num_lr_decays=num_lr_decays[3],
       step_size=10,
       scaler_type='standard', random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=10,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%10,step_size=10)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE}")

INFO:lightning_fabric.utilities.seed:Global seed set to 9583243


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

...

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Random Seed: 104272893
RMSE: 0.0035719699004541122 | QLIKE: 0.04615282962756616 | MAE: 0.0023915991758405673


In [ ]:
for m in range (5):
  randomseed=randrange(129228148)
  model=TFT(h=10, input_size=84, stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=300,
       n_head=10, attn_dropout=0.3, dropout=0.4, loss=MSE_2DWD(),
       max_steps=250, learning_rate=learning_rate[3],val_check_steps=700,
       num_lr_decays=num_lr_decays[3],
       step_size=10,
       scaler_type='standard', random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=10,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%10,step_size=10)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE}")

INFO:lightning_fabric.utilities.seed:Global seed set to 96656312


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

...

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Random Seed: 30728644
RMSE: 0.003550671339403769 | QLIKE: 0.04581196475573554 | MAE: 0.0023534821608740407


# Test

In [8]:
test_length=int(len(df['AMT'])*0.3)
df1=df.unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

In [9]:
model=TFT(h=22, input_size=84, stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=300,
       n_head=10, attn_dropout=0.3, dropout=0.4, loss=MSE_2DWD(),
       max_steps=250, learning_rate=learning_rate[3],val_check_steps=700,
       num_lr_decays=num_lr_decays[3],
       step_size=22,
       scaler_type='standard',
              random_seed=108266631)
fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
forecasts = fcst.cross_validation(df=df1,val_size=22,static_df=df_static,n_windows=None, test_size=test_length-test_length%22,step_size=22)
forecasts = forecasts.dropna()
Y_hat=forecasts["TFT"].values
Y_true=forecasts["y"].values
RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAPE: {MAE}")

INFO:lightning_fabric.utilities.seed:Global seed set to 108266631


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

RMSE: 0.0064591851785611885 | QLIKE: 0.08141680620140251 | MAPE: 0.003824280711258184


In [10]:
df2=forecasts.drop(columns=["cutoff", "y"]).set_index("ds")
df2=df2.pivot(columns="unique_id")

In [11]:
df2.to_excel("TFT_2DWD forecasts.xlsx")